# Example usage on HPE

### 1. Connect to the spark session

### 2. Download the source code by the procedure of [README.md](../README.md)

In [11]:
%%bash

svn checkout https://github.com/HinnyTsang/pysparkchannel/trunk/pysparkchannel

Checked out revision 8.


### 3. Run the following cells

The example will regenerate `module_a` and `module_b` in `dest` directory.

In [10]:
# %%local
# Run this cell in local

# Import the pysparkchannel module
import os
import shutil
import sys
sys.path.append('..')

import importlib
from pysparkchannel import core 
importlib.reload(core)
ModuleParser = core.ModuleParser

# Remove file if exist
if os.path.exists("dist"):
    shutil.rmtree("dist", ignore_errors=True)

# Create the parser
parser = core.ModuleParser(verbose = False)

# Parse the cus
# tom module one by one.
parser \
    .parse_module("sample/module_a") \
    .parse_module("sample/module_b") \
    .module_to_json()

# Generate script to rewrite the module on spark cluster
parsed_modules = parser.parsed_modules
rebuild_script = parser.generate_script()

### 4. Use spark magic to copy the script to the spark session

In [11]:
# %%send_to_spark -i parsed_modules -t str -n parsed_modules

In [12]:
# %%send_to_spark -i rebuild_script -t str -n rebuild_script

### 5. Run the script on the spark session to regenerate the modules

In [14]:
# %%spark

with open("parsed_modules.json", "w", encoding="utf-8") as file:
    file.write(parser.parsed_modules) 

# Execute the script to rebuild the modules
exec(rebuild_script)

Reconstructing folder test/sample/module_a
Reconstructing file test/sample/module_a/example_table_out.xlsx
Reconstructing file test/sample/module_a/__init__.py
Reconstructing file test/sample/module_a/example_table.xlsx
Reconstructing folder test/sample/module_b
Reconstructing folder test/sample/module_b/sub_module_i
Reconstructing file test/sample/module_b/sub_module_i/sub_module_ii.py
Reconstructing file test/sample/module_b/sub_module_i/__init__.py
Reconstructing file test/sample/module_b/module_bi.py
Reconstructing file test/sample/module_b/__init__.py
